# An Introduction to Models in Pyro

The basic unit of probabilistic programs is the _stochastic function_. 
This is an arbitrary Python callable that combines two ingredients:

- deterministic Python code; and
- primitive stochastic functions that call a random number generator

Concretely, a stochastic function can be any Python object with a `__call__()` method, like a function, a method, or a PyTorch `nn.Module`.

Throughout the tutorials and documentation, we will often call stochastic functions *models*, since stochastic functions can be used to represent simplified or abstract descriptions of a process by which data are generated.  Expressing models as stochastic functions means that models can be composed, reused, imported, and serialized just like regular Python callables. 

# Pyro的Model

概率程序的基本单元是_随机函数_。它可以是任意python的callable对象，其包含两部分：

- 拥有确定性输出的Python代码，以及
- 调用随机数生成器的primitive(基元)随机函数

具体而言，任何拥有`__call__()`方法的Python对象都可以作为随机函数，如Python的函数，方法或者PyTorch的`nn.Module`。

在整个教程和文档中，我们经常称随机函数为*Model*（模型），因为随机函数可以用来抽象或简化地表示一些生成数据的过程。
将model表示为随机函数意味着model可以被复合，复用，导入或者连着用，就像普通的Python callable对象一样。

In [1]:
import torch
import pyro

pyro.set_rng_seed(101)

## Primitive Stochastic Functions

Primitive stochastic functions, or distributions, are an important class of stochastic functions for which we can explicitly compute the probability of the outputs given the inputs.  As of PyTorch 0.4 and Pyro 0.2, Pyro uses PyTorch's [distribution library](http://pytorch.org/docs/master/distributions.html). You can also create custom distributions using [transforms](http://pytorch.org/docs/master/distributions.html#module-torch.distributions.transforms).

Using primitive stochastic functions is easy. For example, to draw a sample `x` from the unit normal distribution $\mathcal{N}(0,1)$ we do the following:

## primitive随机函数

Primitive随机函数，或者说分布，是一种重要的随机函数，特别地我们可以计算它们的输出的概率。在PyTorch 0.4与Pyro 0.2版本，Pyro使用PyTorch
的 [distribution库](http://pytorch.org/docs/master/distributions.html)。你也可以构建自己的类，通过使用[transforms](http://pytorch.org/docs/master/distributions.html#module-torch.distributions.transforms)。

使用primitive随机函数十分简单。作为例子，为了从一个标准正态分布 $\mathcal{N}(0,1)$中采样，我们可以这么做：

In [2]:
loc = 0.   # mean zero
scale = 1. # unit variance
normal = torch.distributions.Normal(loc, scale) # create a normal distribution object
x = normal.rsample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(-1.3905)
log prob tensor(-1.8857)


Here, `torch.distributions.Normal` is an instance of the `Distribution` class that takes parameters and provides sample and score methods. Pyro's distribution library `pyro.distributions` is a thin wrapper around `torch.distributions` because we want to make use of PyTorch's fast tensor math and autograd capabilities during inference.

此处，`torch.distributions.Normal`是一个`Distribution`类的实例，该类接收分布参数，对外提供采样和score方法。注意Pyro的分布库
`pyro.distributions`只是`torch.distributions`的一个浅浅的封装，从而我们可以在推断时利用PyTorch的快速张量计算和自动求导机制。

## A Simple Model

All probabilistic programs are built up by composing primitive stochastic functions and deterministic computation. Since we're ultimately interested in probabilistic programming because we want to model things in the real world, let's start with a model of something concrete. 

Let's suppose we have a bunch of data with daily mean temperatures and cloud cover. We want to reason about how temperature interacts with whether it was sunny or cloudy. A simple stochastic function that describes how that data might have been generated is given by:

## 一个简单的模型

所有概率程序都是以一些primitive随机函数和一些确定性的计算构成的。为了利用概率编程建模现实中的事物，让我们以一个具体的例子作为起点。

让我们假定我们有一系列关于各日的平均温度和多云与否的数据。我们想要研究温度与多云与否的关系。一个简单的描述数据可能的产生方式的随机函数
可以被如此描述：

In [3]:
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()

Let's go through this line-by-line. First, in lines 2-3 we define a binary random variable 'cloudy', which is given by a draw from the bernoulli distribution with a parameter of `0.3`. Since the bernoulli distributions returns `0`s or `1`s, in line 4 we convert the value `cloudy` to a string so that return values of `weather` are easier to parse. So according to this model 30% of the time it's cloudy and 70% of the time it's sunny.

In lines 5-6 we define the parameters we're going to use to sample the temperature in lines 7-9. These parameters depend on the particular value of `cloudy` we sampled in line 2. For example, the mean temperature is 55 degrees (Fahrenheit) on cloudy days and 75 degrees on sunny days. Finally we return the two values `cloudy` and `temp` in line 10.

However, `weather` is entirely independent of Pyro - it only calls PyTorch. We need to turn it into a Pyro program if we want to use this model for anything other than sampling fake data.

让我们一行一行来研究这些代码。首先在第2-3行，我们定义了二值随机变量'cloudy'，它以一个取参数为`0.3`的的伯努利分布被给定。
因为伯努利分布返回`0`或`1`，在第四行我们将值`cloudy`转换为一个字符串，从而`weather`的返回值会比较容易解析。由此，根据这个模型，
有30%的概率取多云，有70%概率取晴天。

在第5-6行，我们定义了之后采样温度时（第7-9行）用的参数。这些参数怎么取依赖于`cloudy`在第二行上获得的值。
作为例子，在多云天时平均温度将为55华氏度，而在晴天为75华氏度。最后我们在第10行返回`cloudy`和`temp`两个值。

## The `pyro.sample` Primitive

To turn `weather` into a Pyro program, we'll replace the `torch.distribution`s with `pyro.distribution`s and the `.sample()` and `.rsample()` calls with calls to `pyro.sample`, one of the core language primitives in Pyro. Using `pyro.sample` is as simple as calling a primitive stochastic function with one important difference:

## `pyro.sample` Primitive

为了将`weather`转为Pyro程序，我们将把`torch.distributions`替换为`pyro.distributions`以及将`.sample()`与`.rsample()`替换为
`pyro.sample`，其为pyro中的核心语言primitive的一部分。使用`pyro.sample`与直接调用primitive随机函数几乎一样简单，除了一个
重要的区别：

In [4]:
x = pyro.sample("my_sample", pyro.distributions.Normal(loc, scale))
print(x)

tensor(-0.8152)


Just like a direct call to `torch.distributions.Normal().rsample()`, this returns a sample from the unit normal distribution. The crucial difference is that this sample is _named_. Pyro's backend uses these names to uniquely identify sample statements and _change their behavior at runtime_ depending on how the enclosing stochastic function is being used. As we will see, this is how Pyro can implement the various manipulations that underlie inference algorithms.

就像直接调用`torch.distributions.Normal().rsample()`一样，这个操作会从标准正态分布采样一个样本。主要区别在于这里样本是 _命名_ 的。
Pyro内部使用这些名字取唯一的区分这些采样语句，并且_在运行时改变它们的行为_，具体怎么改变取决于

Now that we've introduced `pyro.sample` and `pyro.distributions` we can rewrite our simple model as a Pyro program:



现在我们来介绍 `pyro.sample` 与 `pyro.distributions`，我们可以把这个简单模型写成Pyro程序：

In [5]:
def weather():
    cloudy = pyro.sample('cloudy', pyro.distributions.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

for _ in range(3):
    print(weather())

('cloudy', 64.5440444946289)
('sunny', 94.37557983398438)
('sunny', 72.5186767578125)


Procedurally, `weather()` is still a non-deterministic Python callable that returns two random samples. Because the randomness is now invoked with `pyro.sample`, however, it is much more than that. In particular `weather()` specifies a joint probability distribution over two named random variables: `cloudy` and `temp`. As such, it defines a probabilistic model that we can reason about using the techniques of probability theory. For example we might ask: if I observe a temperature of 70 degrees, how likely is it to be cloudy? How to formulate and answer these kinds of questions will be the subject of the next tutorial.

程序上地说，`weather()`仍然是非确定性的Python callble对象，因为它返回两个随机样本。注意现在随机的部分由`pyro.sample`发起，而且该语句
也不止做了这个。特别地，`weather()`指定了一个联合概率分布在两个命名随机变量上`cloudy`和`temp`。运用概率论技术，
我们可以以这个分布进行推断。作为例子我们可能会问：如果我观测到70华氏度的温度，那么处于多云天气的可能性如何呢？如何形式化和
回答这个问题将是下一个教程的主题。

## Universality: Stochastic Recursion, Higher-order Stochastic Functions, and Random Control Flow

We've now seen how to define a simple model. Building off of it is easy. For example:

## 一般性：随机递归，高阶随机函数与随机控制流

我们现在已经看到如何定义单一的简单模型。以这种简单模型作为基础构建更复杂的模型也很容易做到，作为例子：

In [6]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream

This kind of modularity, familiar to any programmer, is obviously very powerful. But is it powerful enough to encompass all the different kinds of models we'd like to express?

It turns out that because Pyro is embedded in Python, stochastic functions can contain arbitrarily complex deterministic Python and randomness can freely affect control flow. For example, we can construct recursive functions that terminate their recursion nondeterministically, provided we take care to pass `pyro.sample` unique sample names whenever it's called. For example we can define a geometric distribution that counts the number of failures until the first success like so:

这种模块化的写法是被很多程序员所熟悉的，显然它会很强大。但是它似乎强大到可以覆盖所有我们想要表达的模型吗?

之后可以看到，由于pyro是内嵌于python中的，随机函数可以包含任意复杂的确定性python代码，而随机性可以自由的影响控制流。
作为例子，我们可以构建递归函数，它以非确定性的方式终止，只要我们始终在调用`pyro.sample`时始终给它唯一的样本名字。
作为例子，我们可以定义一个几何分布，其计数失败次数直到第一次成功，如下：

In [7]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)
    
print(geometric(0.5))

0


Note that the names `x_0`, `x_1`, etc., in `geometric()` are generated dynamically and that different executions can have different numbers of named random variables. 

We are also free to define stochastic functions that accept as input or produce as output other stochastic functions:

注意`geometric()`内的`x_0`,`x_1`之类的名字是自动生成的，而不同的执行则可能产生不同数量的命名随机变量。

我们也很容易定义一些接受其他随机函数为参数或输出随机函数作为返回值的随机函数。

In [8]:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", pyro.distributions.Normal(loc, scale))
    z2 = pyro.sample("z2", pyro.distributions.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

print(make_normal_normal()(1.))

tensor(2.1493)


Here `make_normal_normal()` is a stochastic function that takes one argument and which, upon execution, generates three named random variables.

The fact that Pyro supports arbitrary Python code like this&mdash;iteration, recursion, higher-order functions, etc.&mdash;in conjuction with random control flow means that Pyro stochastic functions are _universal_, i.e. they can be used to represent any computable probability distribution. As we will see in subsequent tutorials, this is incredibly powerful. 

It is worth emphasizing that this is one reason why Pyro is built on top of PyTorch: dynamic computational graphs are an important ingredient in allowing for universal models that can benefit from GPU-accelerated tensor math.

这里`make_normal_normal()`的运行结果是一个随机函数，它接收一个参数。在全部执行完后，共产生了三个命名随机变量。

pyro支持任意的python代码，如迭代，递归，高阶函数。其与随机控制流的结合意味着pyro随机函数是 _一般_ 的。
如它们可以被用来表示任意的可计算的分布，正如我们将在下个教程所看到的，这意味着很强的表达能力。

为何pyro要建立在pytorch之上呢？动态计算图对于一般模型而言是一个重要成分，它使得一般模型可以受益于GPU加速的tensor计算。

## Next Steps

We've shown how we can use stochastic functions and primitive distributions to represent models in Pyro. In order to learn models from data and reason about them we need to be able to do inference. This is the subject of the [next tutorial](intro_part_ii.ipynb).

## 下一步

我们已经展示了我们可以如何使用随机函数和primitive分布来表示pyro中的模型。为了从数据中建立模型并使用它们，我们需要进行推断。
这是 [下一个教程](intro_part_ii.ipynb)的内容。